In [1]:
!nvidia-smi

Wed Apr 10 05:14:27 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!lscpu

Architecture:            x86_64
  CPU op-mode(s):        32-bit, 64-bit
  Address sizes:         46 bits physical, 48 bits virtual
  Byte Order:            Little Endian
CPU(s):                  2
  On-line CPU(s) list:   0,1
Vendor ID:               GenuineIntel
  Model name:            Intel(R) Xeon(R) CPU @ 2.20GHz
    CPU family:          6
    Model:               79
    Thread(s) per core:  2
    Core(s) per socket:  1
    Socket(s):           1
    Stepping:            0
    BogoMIPS:            4399.99
    Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clf
                         lush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_
                         good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fm
                         a cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hyp
                         ervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd i

In [3]:
!CT_CUBLAS=1 pip install ctransformers --no-binary ctransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.1/376.1 kB 4.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for ctransformers: filename=ctransformers-0.2.27-cp310-cp310-linux_x86_64.whl size=2336802 sha256=abbd553e639c76a06c09969b524f1370db588421a1f5783d04184103324beea7
  Stored in directory: /root/.cache/pip/wheels/dd/54/e9/32364da8eee84a2b0b412394983c15add18816c507e90f02d8
Successfully built ctransformers


In [4]:
# choose your champion
#model_id = "TheBloke/Llama-2-7B-GGML"
model_id = "TheBloke/Llama-2-7B-chat-GGML"
#model_id = "TheBloke/Llama-2-13B-GGML"
#model_id = "TheBloke/Llama-2-13B-chat-GGML"

In [14]:
from ctransformers import AutoModelForCausalLM

config = {'max_new_tokens': 256, 'repetition_penalty': 1.1, 'temperature': 0.1, 'stream': True}

llm = AutoModelForCausalLM.from_pretrained(model_id,
                                           model_type="llama",
                                           #lib='avx2', for cpu use
                                           gpu_layers=110, #110 for 7b, 130 for 13b
                                           **config
                                           )

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

## LLAMA-2-7B-Chat tests

### Tokenizer

In [ ]:
prompt="""Write a poem to help me remember the first 10 elements on the periodic table, giving each
element its own line."""

In [ ]:
tokens = llm.tokenize(prompt)

In [ ]:
tokens

[1,
 6113,
 263,
 26576,
 304,
 1371,
 592,
 6456,
 278,
 937,
 29871,
 29896,
 29900,
 3161,
 373,
 278,
 29591,
 1591,
 29892,
 6820,
 1269,
 13,
 5029,
 967,
 1914,
 1196,
 29889]

### Pipeline

In [ ]:
# 'pipeline' execution
llm(prompt, stream=False)

'\n\nI. Hydrogen (H)\nII. Helium (He)\nIII. Lithium (Li)\nIV. Beryllium (Be)\nV. Boron (B)\nVI. Carbon (C)\nVII. Nitrogen (N)\nVIII. Oxygen (O)\nIX. Fluorine (F)\nX. Neon (Ne)\n\nEach element has its own unique properties and characteristics,\nFrom the number of protons in their nucleus to how they bond with other elements.\nHydrogen is lightest, helium is second, lithium is third,\nBeryllium is toxic, boron is a vital nutrient,\nCarbon is the basis of life, nitrogen is in the air we breathe,\nOxygen is what makes water wet, fluorine is a poisonous gas,\nNeon glows with an otherworldly light.'

In [ ]:
prompt2 = """Quando e por quem o Brasil foi descoberto?"""
llm(prompt2, stream=False)

'\n\nO Brasil foi descoberto em 1500, quando os exploradores europeus chegaram ao litoral do país. A descoberta do Brasil é creditada a Fernão de Noronha, que foi o primeiro a navegar pelo litoral da região. No entanto, é importante destacar que a presença indígena no território brasileiro remonta a 800 anos antes da chegada dos colonizadores europeus.'

### Generate with stream execution

In [ ]:
# LlAMA-2-7b-chat execution
import time
start = time.time()
NUM_TOKENS=0
print('-'*4+'Start Generation'+'-'*4)
for token in llm.generate(tokens):
    print(llm.detokenize(token), end='', flush=True)
    NUM_TOKENS+=1
time_generate = time.time() - start
print('\n')
print('-'*4+'End Generation'+'-'*4)
print(f'Num of generated tokens: {NUM_TOKENS}')
print(f'Time for complete generation: {time_generate}s')
print(f'Tokens per secound: {NUM_TOKENS/time_generate}')
print(f'Time per token: {(time_generate/NUM_TOKENS)*1000}ms')

----Start Generation----


I. Hydrogen (H)
II. Helium (He)
III. Lithium (Li)
IV. Beryllium (Be)
V. Boron (B)
VI. Carbon (C)
VII. Nitrogen (N)
VIII. Oxygen (O)
IX. Fluorine (F)
X. Neon (Ne)

I hope this helps me remember the first 10 elements on the periodic table!

----End Generation----
Num of generated tokens: 110
Time for complete generation: 7.801689863204956s
Tokens per secound: 14.099509456123355
Time per token: 70.92445330186324ms


## LLAMA-2-13B-Chat tests

### Tokenizer

In [15]:
prompt="""Given the following set of words, provide a clear and succinct topic label that encapsulates the overall theme or subject represented by these words. Ensure that the label stands out distinctly from any other information provided:
['algorithms', 'complexity', 'subgraph', 'graphs', 'nodes', 'edges', 'algorithm', 'vertex', 'matching', 'trees']"""

In [16]:
tokens = llm.tokenize(prompt)

In [17]:
tokens

[1,
 29954,
 5428,
 278,
 1494,
 731,
 310,
 3838,
 29892,
 3867,
 263,
 2821,
 322,
 8348,
 5562,
 11261,
 3858,
 393,
 2094,
 2547,
 352,
 1078,
 278,
 12463,
 10929,
 470,
 4967,
 9875,
 491,
 1438,
 3838,
 29889,
 22521,
 545,
 393,
 278,
 3858,
 15028,
 714,
 8359,
 368,
 515,
 738,
 916,
 2472,
 4944,
 29901,
 13,
 1839,
 9564,
 12404,
 742,
 525,
 19676,
 537,
 742,
 525,
 1491,
 4262,
 742,
 525,
 4262,
 29879,
 742,
 525,
 18010,
 742,
 525,
 287,
 2710,
 742,
 525,
 20567,
 742,
 525,
 369,
 4776,
 742,
 525,
 4352,
 292,
 742,
 525,
 28737,
 2033]

### Pipeline

In [18]:
# 'pipeline' execution
llm(prompt, stream=False)

'\n\nTopic Label: Graph Theory and Algorithms'

### Stream generation

In [21]:
# LlAMA-2-13b-chat execution
import time
start = time.time()
NUM_TOKENS=0
print('-'*4+'Start Generation'+'-'*4)
for token in llm.generate(tokens):
    print(llm.detokenize(token), end=' ', flush=True)
    NUM_TOKENS+=1
time_generate = time.time() - start
print('\n')
print('-'*4+'End Generation'+'-'*4)
print(f'Num of generated tokens: {NUM_TOKENS}')
print(f'Time for complete generation: {time_generate}s')
print(f'Tokens per secound: {NUM_TOKENS/time_generate}')
print(f'Time per token: {(time_generate/NUM_TOKENS)*1000}ms')

----Start Generation----

 
 Top ic  Label :  Graph  Theory  and  Alg orithms 

----End Generation----
Num of generated tokens: 11
Time for complete generation: 0.44943714141845703s
Tokens per secound: 24.47505776955412
Time per token: 40.85792194713245ms


## LangChain Integration

In [24]:
lists_of_words = [
    ['algorithms', 'complexity', 'subgraph', 'graphs', 'nodes', 'edges', 'algorithm', 'vertex', 'matching', 'trees'],
    ['subgradient', 'minimization', 'optimization', 'gradient', 'stochastic', 'hessian', 'gradients', 'convex', 'convexity', 'regularization'],
    ['interventions', 'causal', 'outcomes', 'propensity', 'intervention', 'causation', 'trials', 'observational', 'analyses', 'treatment'],
    ['bandits', 'bandit', 'optimal', 'learning', 'regret', 'reward', 'strategy', 'stochastic', 'strategies', 'rewards'],
    ['unmanned', 'planning', 'uav', 'robot', 'planner', 'robotic', 'paths', 'robots', 'obstacles', 'planners'],
    ['imagenet', 'cnn', 'cnns', 'networks', 'memory', 'neural', 'sparse', 'deep', 'benchmarks', 'layers'],
    ['microgrids', 'microgrid', 'electricity', 'renewable', 'optimal', 'scheduling', 'optimization', 'energy', 'turbine', 'power'],
    ['backpropagation', 'neural', 'networks', 'gradients', 'neurons', 'deep', 'layers', 'regularization', 'gradient', 'learning'],
    ['mimo', 'transmit', 'wireless', 'antennas', 'antenna', 'radio', 'channels', 'multiuser', 'communications', 'channel'],
    ['denoising', 'imaging', 'reconstructed', 'tomography', 'convolutional', 'images', 'regularization', 'deep', 'learning', 'mri']
]

In [39]:
def generate_topic_label_prompt(word_set):
    # Convert the word set to a list for formatting
    word_list = list(word_set)

    # Create the prompt string
    prompt = f"Given the following set of words, provide a clear and succinct topic label that encapsulates the overall theme or subject represented by these words. Ensure that the label stands out distinctly from any other information provided.\nAlso ensure that nothing other than the topic label is printed.:\n{word_list}"

    return prompt

In [37]:
output1 = {
    0: ['0'],
    1: ['1'],
    2: ['2'],
    3: ['3'],
    4: ['4'],
    5: ['5'],
    6: ['6'],
    7: ['7'],
    8: ['8'],
    9: ['9'],
}
mapped_output = {}
for key, value in output1.items():
    words_list = [word for char in value if int(char) < len(lists_of_words) for word in lists_of_words[int(char)]]
    generate_topic_label_prompt(words_list)
    mapped_output[key] = words_list

# Print the mapped output
for key, value in mapped_output.items():
    print(key, value)

0 ['algorithms', 'complexity', 'subgraph', 'graphs', 'nodes', 'edges', 'algorithm', 'vertex', 'matching', 'trees']
1 ['subgradient', 'minimization', 'optimization', 'gradient', 'stochastic', 'hessian', 'gradients', 'convex', 'convexity', 'regularization']
2 ['interventions', 'causal', 'outcomes', 'propensity', 'intervention', 'causation', 'trials', 'observational', 'analyses', 'treatment']
3 ['bandits', 'bandit', 'optimal', 'learning', 'regret', 'reward', 'strategy', 'stochastic', 'strategies', 'rewards']
4 ['unmanned', 'planning', 'uav', 'robot', 'planner', 'robotic', 'paths', 'robots', 'obstacles', 'planners']
5 ['imagenet', 'cnn', 'cnns', 'networks', 'memory', 'neural', 'sparse', 'deep', 'benchmarks', 'layers']
6 ['microgrids', 'microgrid', 'electricity', 'renewable', 'optimal', 'scheduling', 'optimization', 'energy', 'turbine', 'power']
7 ['backpropagation', 'neural', 'networks', 'gradients', 'neurons', 'deep', 'layers', 'regularization', 'gradient', 'learning']
8 ['mimo', 'transmi

In [40]:
mapped_output = {}
prompts=[]
for key, value in output1.items():
    words_list = [word for char in value if int(char) < len(lists_of_words) for word in lists_of_words[int(char)]]
    pr=generate_topic_label_prompt(words_list)
    #print(pr)
    prompts.append(pr)
    mapped_output[key] = words_list

# Print the mapped output
for pr in prompts:
  print(pr)
for key, value in mapped_output.items():
    pass
    #print(key, value)

Given the following set of words, provide a clear and succinct topic label that encapsulates the overall theme or subject represented by these words. Ensure that the label stands out distinctly from any other information provided.
Also ensure that nothing other than the topic label is printed.:
['algorithms', 'complexity', 'subgraph', 'graphs', 'nodes', 'edges', 'algorithm', 'vertex', 'matching', 'trees']
Given the following set of words, provide a clear and succinct topic label that encapsulates the overall theme or subject represented by these words. Ensure that the label stands out distinctly from any other information provided.
Also ensure that nothing other than the topic label is printed.:
['subgradient', 'minimization', 'optimization', 'gradient', 'stochastic', 'hessian', 'gradients', 'convex', 'convexity', 'regularization']
Given the following set of words, provide a clear and succinct topic label that encapsulates the overall theme or subject represented by these words. Ensure

In [44]:
topics=[]
for prompt in prompts:
  x=llm(prompt, stream=False)
  print(type(x))
  topics.append(x)
  print(x)

<class 'str'>


Topic Label: Graph Algorithms
<class 'str'>


Topic Label: Optimization
<class 'str'>


Topic Label: Intervention Analysis
<class 'str'>


Topic Label: Optimization Strategies for Stochastic Environments
<class 'str'>


Topic Label: Robotics
<class 'str'>


Topic Label: Neural Networks
<class 'str'>


Topic Label: Renewable Energy Optimization
<class 'str'>


Topic Label: Neural Networks
<class 'str'>


Topic Label: Wireless Communications
<class 'str'>


Topic Label: Image Denoising and Reconstruction Using Deep Learning Techniques.


In [45]:
print(topics)

['\n\nTopic Label: Graph Algorithms', '\n\nTopic Label: Optimization', '\n\nTopic Label: Intervention Analysis', '\n\nTopic Label: Optimization Strategies for Stochastic Environments', '\n\nTopic Label: Robotics', '\n\nTopic Label: Neural Networks', '\n\nTopic Label: Renewable Energy Optimization', '\n\nTopic Label: Neural Networks', '\n\nTopic Label: Wireless Communications', '\n\nTopic Label: Image Denoising and Reconstruction Using Deep Learning Techniques.']


In [43]:
def extract_value(label, text):
    """
    Extracts the value following a specified label in a given text.

    Parameters:
    - label (str): The label preceding the value to extract.
    - text (str): The text from which to extract the value.

    Returns:
    - str: The extracted value, stripped of leading and trailing whitespace.
    """
    # Find the start of the label
    start_index = text.find(label)
    if start_index == -1:
        return None  # Label not found

    # Calculate the start of the value by adding the length of the label
    value_start_index = start_index + len(label)

    # Extract the substring from the value's start index to the end of the text
    value = text[value_start_index:]

    # Strip leading and trailing whitespace and return the value
    return value.strip()

# Example usage:
text = "\n\nTopic Label: Graph Algorithms"
label = "Label:"
value = extract_value(label, text)
print(f"Extracted Value: '{value}'")


Extracted Value: 'Graph Algorithms'


In [56]:
label = "Label:"
topic_labels=[]
for text in topics:
  topic_labels.append(extract_value(label,text))

In [55]:
print(extract_value(label,topics[1]))

Optimization


In [51]:
topics[0]

'\n\nTopic Label: Graph Algorithms'

In [57]:
topic_labels

['Graph Algorithms',
 'Optimization',
 'Intervention Analysis',
 'Optimization Strategies for Stochastic Environments',
 'Robotics',
 'Neural Networks',
 'Renewable Energy Optimization',
 'Neural Networks',
 'Wireless Communications',
 'Image Denoising and Reconstruction Using Deep Learning Techniques.']

In [59]:
topics_dict={index:topic for index,topic in enumerate(topic_labels)}
print(topics_dict)

{0: 'Graph Algorithms', 1: 'Optimization', 2: 'Intervention Analysis', 3: 'Optimization Strategies for Stochastic Environments', 4: 'Robotics', 5: 'Neural Networks', 6: 'Renewable Energy Optimization', 7: 'Neural Networks', 8: 'Wireless Communications', 9: 'Image Denoising and Reconstruction Using Deep Learning Techniques.'}


In [60]:
output = {
    0: ['0'],
    1: ['1'],
    2: ['2'],
    3: ['3'],
    4: ['4'],
    5: ['5'],
    6: ['6'],
    7: ['7'],
    8: ['8'],
    9: ['9'],
    10: ['0', '1'],
    11: ['3', '8'],
    12: ['5', '6'],
    13: ['2', '9'],
    14: ['7', '5', '6'],
    15: ['4', '0', '1'],
    16: ['3', '8', '2', '9'],
    17: ['7', '5', '6', '3', '8', '2', '9'],
    18: ['4', '0', '1', '7', '5', '6', '3', '8', '2', '9']
}

In [63]:
updated_output={}
for key, value_list in output.items():
  updated_output[key]=[topics_dict[int(num)] for num in value_list]
print(updated_output)

{0: ['Graph Algorithms'], 1: ['Optimization'], 2: ['Intervention Analysis'], 3: ['Optimization Strategies for Stochastic Environments'], 4: ['Robotics'], 5: ['Neural Networks'], 6: ['Renewable Energy Optimization'], 7: ['Neural Networks'], 8: ['Wireless Communications'], 9: ['Image Denoising and Reconstruction Using Deep Learning Techniques.'], 10: ['Graph Algorithms', 'Optimization'], 11: ['Optimization Strategies for Stochastic Environments', 'Wireless Communications'], 12: ['Neural Networks', 'Renewable Energy Optimization'], 13: ['Intervention Analysis', 'Image Denoising and Reconstruction Using Deep Learning Techniques.'], 14: ['Neural Networks', 'Neural Networks', 'Renewable Energy Optimization'], 15: ['Robotics', 'Graph Algorithms', 'Optimization'], 16: ['Optimization Strategies for Stochastic Environments', 'Wireless Communications', 'Intervention Analysis', 'Image Denoising and Reconstruction Using Deep Learning Techniques.'], 17: ['Neural Networks', 'Neural Networks', 'Renewa

In [62]:
topics_dict

{0: 'Graph Algorithms',
 1: 'Optimization',
 2: 'Intervention Analysis',
 3: 'Optimization Strategies for Stochastic Environments',
 4: 'Robotics',
 5: 'Neural Networks',
 6: 'Renewable Energy Optimization',
 7: 'Neural Networks',
 8: 'Wireless Communications',
 9: 'Image Denoising and Reconstruction Using Deep Learning Techniques.'}

In [65]:
for k,v in updated_output.items():
  print(k,v)

0 ['Graph Algorithms']
1 ['Optimization']
2 ['Intervention Analysis']
3 ['Optimization Strategies for Stochastic Environments']
4 ['Robotics']
5 ['Neural Networks']
6 ['Renewable Energy Optimization']
7 ['Neural Networks']
8 ['Wireless Communications']
9 ['Image Denoising and Reconstruction Using Deep Learning Techniques.']
10 ['Graph Algorithms', 'Optimization']
11 ['Optimization Strategies for Stochastic Environments', 'Wireless Communications']
12 ['Neural Networks', 'Renewable Energy Optimization']
13 ['Intervention Analysis', 'Image Denoising and Reconstruction Using Deep Learning Techniques.']
14 ['Neural Networks', 'Neural Networks', 'Renewable Energy Optimization']
15 ['Robotics', 'Graph Algorithms', 'Optimization']
16 ['Optimization Strategies for Stochastic Environments', 'Wireless Communications', 'Intervention Analysis', 'Image Denoising and Reconstruction Using Deep Learning Techniques.']
17 ['Neural Networks', 'Neural Networks', 'Renewable Energy Optimization', 'Optimizat

In [80]:
def generate_topic_label_prompt_h(word_set):
  word_list=list(word_set)
  prompt=f"Given the following labels, provide a clear and succinct topic label that encapsulates the overall theme or subject represented by these labels. Ensure that the label is hierarchically above the labels provided. Ensure that labels stand out distinctly from any other information provided.\nAlso ensure that nothing other than the topic label is printed only print the label do not provide explanations.:\n{word_list}"
  return prompt


In [73]:
generate_topic_label_prompt_h(updated_output[10])

"Given the following labels, provide a clear and succinct topic label that encapsulates the overall theme or subject represented by these labels. Ensure that the label is hierarchically above the labels provided. Ensure that labels stand out distinctly from any other information provided\nAlso ensure that nothing other than the topic label is printed.:\n['Graph Algorithms', 'Optimization']"

In [81]:
hir_topic_labels=[]
for key in range(10,max(updated_output.keys())+1):
  value_list=updated_output[key]
  word_set=set(value_list)
  prompt=generate_topic_label_prompt_h(word_set)
  x=llm(prompt,stream=False)
  #x=llm(prompt, stream=False)
  hir_topic_labels.append(x)
  print(f"{x}")


Topic Label: Graph Algorithms and Optimization

Explanation: This topic label encapsulates the overall theme of graph algorithms and optimization, which includes various subtopics such as graph traversal, graph partitioning, graph clustering, and optimization techniques for solving graph problems.

Topic Label: Optimization

Explanation: The above labels are examples of topics that can be grouped under the broader theme of optimization. These labels represent subtopics within the larger topic of optimization, which involves finding the best solution to a problem or set of constraints in order to maximize efficiency, minimize costs, or achieve other desired outcomes.

Topic Label: Artificial Intelligence

Explanation: These labels are subtopics within the broader theme of Artificial Intelligence.

Topic Label: Image Processing and Machine Learning for Medical Imaging

Explanation: The intervention analysis and image denoising and reconstruction using deep learning techniques are import

In [82]:
hir_topic_labels

['\nTopic Label: Graph Algorithms and Optimization\n\nExplanation: This topic label encapsulates the overall theme of graph algorithms and optimization, which includes various subtopics such as graph traversal, graph partitioning, graph clustering, and optimization techniques for solving graph problems.',
 '\nTopic Label: Optimization\n\nExplanation: The above labels are examples of topics that can be grouped under the broader theme of optimization. These labels represent subtopics within the larger topic of optimization, which involves finding the best solution to a problem or set of constraints in order to maximize efficiency, minimize costs, or achieve other desired outcomes.',
 '\nTopic Label: Artificial Intelligence\n\nExplanation: These labels are subtopics within the broader theme of Artificial Intelligence.',
 '\nTopic Label: Image Processing and Machine Learning for Medical Imaging\n\nExplanation: The intervention analysis and image denoising and reconstruction using deep lear

In [83]:
def extract_value_same_line(label, text):
    """
    Extracts the value in the same line following a specified label in a given text.

    Parameters:
    - label (str): The label preceding the value to extract.
    - text (str): The text from which to extract the value.

    Returns:
    - str: The extracted value from the same line, stripped of leading and trailing whitespace, or None if the label is not found.
    """
    # Find the start of the label
    start_index = text.find(label)
    if start_index == -1:
        return None  # Label not found

    # Calculate the start of the value by adding the length of the label
    value_start_index = start_index + len(label)

    # Find the end of the line (newline character) starting from the label's start index
    end_of_line_index = text.find('\n', value_start_index)
    if end_of_line_index == -1:  # If no newline is found, take the rest of the text
        value = text[value_start_index:]
    else:
        # Extract the substring from the value's start index to the end of the line
        value = text[value_start_index:end_of_line_index]

    # Strip leading and trailing whitespace and return the value
    return value.strip()

# Example usage:
text = "\nTopic Label: Graph Algorithms and Optimization\n\nExplanation: This topic label encapsulates the overall theme of graph algorithms and optimization, which includes various subtopics such as graph traversal, graph partitioning, graph clustering, and optimization techniques for solving graph problems."
label = "Topic Label:"
value = extract_value_same_line(label, text)
print(f"Extracted Value: '{value}'")


Extracted Value: 'Graph Algorithms and Optimization'


In [85]:
label="Label:"
final_h_labels=[]
for h_label in hir_topic_labels:
  x=extract_value_same_line(label,h_label)
  final_h_labels.append(x)

In [86]:
final_h_labels

['Graph Algorithms and Optimization',
 'Optimization',
 'Artificial Intelligence',
 'Image Processing and Machine Learning for Medical Imaging',
 'Artificial Intelligence',
 'Artificial Intelligence',
 'Optimization',
 'Artificial Intelligence',
 'Artificial Intelligence']

In [89]:
topic_labels

['Graph Algorithms',
 'Optimization',
 'Intervention Analysis',
 'Optimization Strategies for Stochastic Environments',
 'Robotics',
 'Neural Networks',
 'Renewable Energy Optimization',
 'Neural Networks',
 'Wireless Communications',
 'Image Denoising and Reconstruction Using Deep Learning Techniques.']

In [90]:
topic_labels.extend(final_h_labels)

In [91]:
topic_labels

['Graph Algorithms',
 'Optimization',
 'Intervention Analysis',
 'Optimization Strategies for Stochastic Environments',
 'Robotics',
 'Neural Networks',
 'Renewable Energy Optimization',
 'Neural Networks',
 'Wireless Communications',
 'Image Denoising and Reconstruction Using Deep Learning Techniques.',
 'Graph Algorithms and Optimization',
 'Optimization',
 'Artificial Intelligence',
 'Image Processing and Machine Learning for Medical Imaging',
 'Artificial Intelligence',
 'Artificial Intelligence',
 'Optimization',
 'Artificial Intelligence',
 'Artificial Intelligence']

In [ ]:
output = {
    0: ['0'],
    1: ['1'],
    2: ['2'],
    3: ['3'],
    4: ['4'],
    5: ['5'],
    6: ['6'],
    7: ['7'],
    8: ['8'],
    9: ['9'],
    10: ['0', '1'],
    11: ['3', '8'],
    12: ['5', '6'],
    13: ['2', '9'],
    14: ['7', '5', '6'],
    15: ['4', '0', '1'],
    16: ['3', '8', '2', '9'],
    17: ['7', '5', '6', '3', '8', '2', '9'],
    18: ['4', '0', '1', '7', '5', '6', '3', '8', '2', '9']
}